![A soccer pitch for an international match.](soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [64]:
# Start your code here!
import pandas as pd
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
import pingouin as pg

In [65]:
# Import csv files
file_women = 'women_results.csv'
file_men = 'men_results.csv'

women_results = pd.read_csv(file_women)
men_results = pd.read_csv(file_men)


In [66]:
#Exploratory data analysis
print(women_results.isnull().sum())
print(men_results.isnull().sum())

print(women_results.info())
print(men_results.info())

print(women_results['tournament'].value_counts())
print(men_results['tournament'].value_counts())

Unnamed: 0    0
date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
dtype: int64
Unnamed: 0    0
date          0
home_team     0
away_team     0
home_score    0
away_score    0
tournament    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4884 non-null   int64 
 1   date        4884 non-null   object
 2   home_team   4884 non-null   object
 3   away_team   4884 non-null   object
 4   home_score  4884 non-null   int64 
 5   away_score  4884 non-null   int64 
 6   tournament  4884 non-null   object
dtypes: int64(3), object(4)
memory usage: 267.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44353 entries, 0 to 44352
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  44353 non-null  int64 

In [67]:
# Filtering data to FIFA World Cup only
women_results_fifa = women_results[women_results['tournament'] == 'FIFA World Cup']
men_results_fifa = men_results[men_results['tournament'] == 'FIFA World Cup']

# Filtering data to since 2002
women_results_fifa["date"] = pd.to_datetime(women_results_fifa["date"])
men_results_fifa["date"] = pd.to_datetime(men_results_fifa["date"])

women_results_fifa = women_results_fifa[women_results_fifa["date"].dt.year >= 2002]
men_results_fifa = men_results_fifa[men_results_fifa["date"].dt.year >= 2002]


In [68]:
# Calculate the goals
women_results_fifa['total_goals'] = women_results_fifa['home_score'] + women_results_fifa['away_score']
men_results_fifa['total_goals'] = men_results_fifa['home_score'] + men_results_fifa['away_score']

In [69]:
# Normality test
_, p_women = shapiro(women_results_fifa["total_goals"])
_, p_men = shapiro(men_results_fifa["total_goals"])

alpha = 0.10
use_ttest = (p_women > alpha) and (p_men > alpha)

if use_ttest:
# T-test (independent, one-sided)
    t_stat, p_val_two_sided = ttest_ind(women_results_fifa["total_goals"], men_results_fifa["total_goals"], equal_var=False)
    p_val = p_val_two_sided / 2 if t_stat > 0 else 1 - (p_val_two_sided / 2)
    print(f"T-test p-value (one-sided): {p_val:.4f}")
    
else:
# Mann-Whitney U test (one-sided)
    u_stat, p_val_two_sided = mannwhitneyu(women_results_fifa["total_goals"], men_results_fifa["total_goals"], alternative="greater")
    p_val = p_val_two_sided
    print(f"Mann-Whitney p-value: {p_val:.4f}")



Mann-Whitney p-value: 0.0051


In [70]:
# Mann-Whitney with pingouin
df_wc = pd.concat([
        women_results_fifa.assign(gender="women"),
        men_results_fifa.assign(gender="men")
])
df_wide = df_wc.pivot(columns='gender', values='total_goals')
    
mwu_result = pg.mwu(
        x=df_wide['women'], 
        y=df_wide['men'], 
        alternative='greater'
)
p_val_pingouin = mwu_result['p-val'].values[0]
print(f"Mann-Whitney-Pingouin p-value: {p_val_pingouin:.4f}")

Mann-Whitney-Pingouin p-value: 0.0051


In [71]:
# Result
result = "reject" if p_val < alpha else "fail to reject"
result_dict = {"p_val": p_val, "result": result}

print(result_dict)

{'p_val': 0.005106609825443641, 'result': 'reject'}
